In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding


In [5]:
raw_datasets=load_dataset('glue','mrpc')


README.md: 0.00B [00:00, ?B/s]

mrpc/train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

mrpc/validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

mrpc/test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [6]:
checkpoint='bert-base-uncased'
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
def tokenize_function(example):
  return tokenizer(example['sentence1'],example['sentence2'],truncation=True)

In [8]:
tokenized_dataset=raw_datasets.map(tokenize_function,batched=True)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [9]:
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)

In [21]:
from transformers import TrainingArguments
training_args=TrainingArguments('test-trainer',report_to="none")

In [22]:
from transformers import AutoModelForSequenceClassification

In [23]:
model=AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
from transformers import Trainer

In [25]:
trainer=Trainer(model,training_args,train_dataset=tokenized_dataset['test'],eval_dataset=tokenized_dataset['validation'],data_collator=data_collator,processing_class=tokenizer)

In [26]:
trainer.train()

Step,Training Loss
500,0.432500


TrainOutput(global_step=648, training_loss=0.3742138750759172, metrics={'train_runtime': 121.8508, 'train_samples_per_second': 42.47, 'train_steps_per_second': 5.318, 'total_flos': 190687167928260.0, 'train_loss': 0.3742138750759172, 'epoch': 3.0})

In [28]:
predictions=trainer.predict(tokenized_dataset['validation'])


In [29]:
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


In [30]:
import numpy as np

In [31]:
preds=np.argmax(predictions.predictions,axis=-1)

In [2]:
!pip install evaluate


  Using cached datasets-4.0.0-py3-none-any.whl.metadata (19 kB)
  Using cached huggingface_hub-0.34.4-py3-none-any.whl.metadata (14 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
Using cached datasets-4.0.0-py3-none-any.whl (494 kB)
Using cached huggingface_hub-0.34.4-py3-none-any.whl (561 kB)
Using cached multiprocess-0.70.16-py312-none-any.whl (146 kB)


In [4]:
metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

NameError: name 'preds' is not defined

In [3]:
import evaluate
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)